In [1]:
from base64 import b64decode, b64encode
from Crypto import Random
import imageio
import cv2
import shutil
import time
import sys, os

import aes
import dwt
from Huff import HuffmanCoding

In [2]:
names = ["a", "h", "v", "d"]

In [3]:
def encryptMain(image, key, mode):
    nonce=["","","",""]
    data = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    coeffs_from_arr = dwt.ProcesoDWT(data)
    dwt.Save_txt(coeffs_from_arr)
    h = HuffmanCoding()
    for cr in range(len(names)):
        path = "{}.txt".format(names[cr])
        h.compress(path)
        n=names[cr]
        with open(f"{n}_c.bin","rb") as file, open(f"{n}_cl.bin","wb") as out:
            plaintext = b64encode(file.read())
            appended = aes.saturar(plaintext)
            ciphertext, nonce[cr]= aes.encrypt(appended,key,mode)
            #ciphertext = aes.encrypt(appended,key,mode)
            out.write(ciphertext)
        os.remove(f"{n}.txt")
        #os.remove(f"{n}_c.bin")
    return nonce

In [11]:
def decryptMain(imageName, key, mode, nonce=None):
    h = HuffmanCoding()
    for cr in range(len(names)):
        n =names[cr]
        with open("{}_cl.bin".format(n),"rb") as file, open("{}_c.bin".format(n),"wb") as out:
            if mode == "ECB":
                decrypted = aes.decrypt(file.read(),key,mode)
            else:
                decrypted = aes.decrypt(file.read(),key,mode, nonce[cr])
            satured= aes.remove_space_padding(decrypted)
            plaintextout = b64decode(satured)
            out.write(plaintextout)
        path = "{}_c.bin".format(names[cr])
        h.decompress(path)
    name_recon = dwt.ReconstructDWT()
    imageio.imwrite('{}_r.png'.format(imageName), name_recon)
    for trash in names:
            os.remove(f"{trash}.txt")
            os.remove(f"{trash}d.txt")
            #os.remove(f"{trash}_c.bin")

In [42]:
dir="Analisis/Images/"
name ="lena.png"
key = Random.new().read(16)

In [43]:
nonce=encryptMain(dir+name,key,"OFB")

47.324 %
8.421 s
48.136 %
8.165 s
48.114 %
8.937 s
48.066 %
9.288 s


In [44]:
key

b'\xdd\xe6N\x04\xa2 f\xc9.\xdc\xe2\n\x9d\x19\xfe\x19'

In [28]:
key=b'\xado\xa8h\n\xfd\x1a8J\xa1\xc3DH.\xab\xa9'

b'\xado\xa8h\n\xfd\x1a8J\xa1\xc3DH.\xab\xa9'

In [12]:
decryptMain("barbara",key,"ECB")

11.337 s
12.315 s
11.871 s
11.739 s


In [5]:
dir="Images/"
for files in os.listdir(dir):
    file = dir+files
    if file[-4:] != ".png":
        continue
    key = Random.new().read(16)
    start=time.time()
    print("I started "+files)
    nonce = encryptMain(file,key, mode="OFB")
    #encryptMain(file,key, mode="ECB")
    midle=time.time()
    try:
        decryptMain(files[:-4],key,mode="OFB", nonce=nonce)
        #decryptMain(files[:-4],key,mode="ECB")
        end= time.time()
    except:
        enc = midle - start
        print(file+" has failed")
        enc = midle - start
        f=open(f"{files[:-4]}.txt","w")
        f.write("Title: "+file+"\n")
        f.write(f"key: {key}\n")
        f.write(f"iv: {nonce}\n")
        f.write(f"Time encrypted: {enc}\n")
        f.close()
        continue
        
    enc = midle - start
    des = end - midle
    
    f=open(f"{files[:-4]}.txt","w")
    f.write("Title: "+file+"\n")
    f.write(f"key: {key}\n")
    f.write(f"iv: {nonce}\n")
    f.write(f"Time encrypted: {enc}\n")
    f.write(f"Time encrypted: {des}\n")
    total=end - start
    f.write(f"Total: {total}")
    f.close()
    if not os.path.exists(file[:-4]):
        os.makedirs(file[:-4])
    for clf in names:
        shutil.move(f"{clf}_cl.bin",file[:-4])
    shutil.move(f"{files[:-4]}_r.png",file[:-4])
    shutil.move(file,file[:-4])
    shutil.move(f"{files[:-4]}.txt",file[:-4])

I started barbara.png
47.446 %
8.224 s
48.161 %
7.673 s
48.165 %
8.364 s
48.157 %
7.187 s
11.386 s
12.184 s
11.915 s
12.229 s


I started color.png
29.397 %
13.528 s
44.416 %
26.018 s
45.655 %
27.445 s
45.654 %
25.354 s
19.792 s
34.561 s
37.72 s
36.411 s


I started eye.png
47.38 %
901.013 s
48.078 %
923.869 s
48.071 %
916.535 s
48.002 %
914.619 s
977.456 s
1040.271 s
1034.876 s
1035.319 s


I started lena.png
47.324 %
8.584 s
48.136 %
8.0 s
48.114 %
7.101 s
48.066 %
8.357 s
11.026 s
13.884 s
13.449 s
14.535 s


I started lenaV2.png
47.066 %
52.214 s
47.663 %
49.834 s
47.757 %
48.995 s
47.468 %
48.601 s
59.601 s
64.686 s
65.894 s
65.064 s


I started logo.png
35.652 %
0.476 s
46.502 %
0.629 s
47.221 %
0.488 s
47.06 %
0.487 s
1.413 s
1.643 s
1.749 s
1.537 s


I started Mandrill.png
47.408 %
9.042 s
48.114 %
9.576 s
48.133 %
7.141 s
48.147 %
9.532 s
14.78 s
15.309 s
12.946 s
12.685 s


I started peppers.png
47.439 %
9.032 s
48.183 %
8.017 s
48.189 %
7.123 s
48.207 %
7.723 s
11.113 s
12.426 s
13.417 s
13.181 s


I started pinguin.png
33.817 %
4.391 s
44.68 %
8.315 s
45.958 %
6.649 s
44.806 %
5.419 s
6.697 s
9.659 s
9.236 s
9.625 s


I started scale.png
47.07 %
4.576 s
47.232 %
4.148 s
47.377 %
3.801 s
47.152 %
3.534 s
6.523 s
7.807 s
7.188 s
8.081 s


In [5]:
import converter
import test

In [6]:
def encryptImage(image, key, mode):
    nonce=["","","",""]
    data = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    shape=data.shape
    coeffs_from_arr = dwt.ProcesoDWT(data)
    dwt.Save_txt(coeffs_from_arr)
    h = HuffmanCoding()
    for cr in range(len(names)):
        path = "{}.txt".format(names[cr])
        h.compress(path)
        n=names[cr]
        with open(f"{n}_c.bin","rb") as file, open(f"{n}_cl.bin","wb") as out:
            plaintext = b64encode(file.read())
            appended = aes.saturar(plaintext)
            #ciphertext, nonce[cr]= aes.encrypt(appended,key,mode)
            ciphertext = aes.encrypt(appended,key,mode)
            out.write(ciphertext)
        with open(f"{n}_c.bin","wb") as file:
            file.write(appended)
        converter.convertImage(f"{n}_c.bin",f"{n}_init",shape[0])
        converter.convertImage(f"{n}_cl.bin",f"{n}_mid",shape[0])
        os.remove(f"{n}.txt")
        os.remove(f"{n}_c.bin")
    if mode == "ECB":
        return shape[0]
    else:
        return nonce, shape[0]

In [7]:
def decryptedImage(name,shape, key,mode,nonce=None):
    names = ["a", "h", "v", "d"]
    for cr in range(len(names)):
        n =names[cr]
        with open("{}_cl.bin".format(n),"rb") as file, open("{}_c.bin".format(n),"wb") as out:
            if mode == "ECB":
                decrypted = aes.decrypt(file.read(),key,mode)
            else:
                decrypted = aes.decrypt(file.read(),key,mode, nonce[cr])
                out.write(decrypted)
        converter.convertImage(f"{n}_c.bin",f"{n}_out",shape)

In [10]:
dir="Images/"
files="barbara.png"
file = dir+files
key = Random.new().read(16)
print("I started "+files)
start=time.time()
#nonce, shape = encryptImage(file,key, mode="OFB")
shape=encryptImage(file,key, mode="ECB")

#decryptedImage(files[:-4],shape,key,mode, nonce)
decryptedImage(files[:-4],shape,key,mode="ECB")
end= time.time()
    
for n in names:
    data=test.testing(n)
print (data)

I started barbara.png
47.446 %
8.244 s


48.161 %
7.586 s


48.165 %
8.162 s


48.157 %
8.059 s


ValueError: Image must be 2D (grayscale, RGB, or RGBA).

In [54]:
dir="Images/"
for files in os.listdir(dir):
    file = dir+files
    if file[-4:] != ".png":
        continue
    key = Random.new().read(16)
    print("I started "+files)
    #nonce, shape = encryptImage(file,key, mode="OFB")
    shape=encryptMain(file,key, mode="ECB")

    #decryptedImage(files[:-4],shape,key,mode="OFB", nonce=nonce)
    decryptMain(files[:-4],shape,key,mode="ECB")

    for n in names:
        data=test.testing(n)
        
    f=open(f"{files[:-4]}.txt","w")
    f.write("Title: "+file+"\n")
    total=end - start
    f.write("Test: "+data)
    f.close()
    if not os.path.exists(file[:-4]):
        os.makedirs(file[:-4])
    for clf in names:
        shutil.move(f"{clf}_cl.bin",file[:-4])
    shutil.move(f"{files[:-4]}_init.png",file[:-4])
    shutil.move(f"{files[:-4]}_mid.png",file[:-4])
    shutil.move(f"{files[:-4]}_out.png",file[:-4])
    shutil.move(file,file[:-4])
    shutil.move(f"{files[:-4]}.txt",file[:-4])

I started barbara.png


KeyboardInterrupt: 

In [14]:
files

'barbara.png'

In [12]:
#modes=["OFB",OCB","CTR","GCM"]
modes=["ECB"]
for mode in modes:
    dir="Images/"
    for files in os.listdir(dir):
        file = dir+files
        if file[-4:] != ".png":
            continue
        key = Random.new().read(16)
        print("I started "+files)
        #nonce, shape = encryptImage(file,key, mode)
        shape=encryptImage(file,key, mode="ECB")

        #decryptedImage(files[:-4],shape,key,mode, nonce)
        decryptedImage(files[:-4],shape,key,mode="ECB")
        
        with open(f"{mode}.txt","a") as f:
            f.write("Title: "+file+"\n") 
            for n in names:
                data=test.testing(n)
                f.write(f"Test: {n}\n")
                f.write(f"{str(data)}")

        if not os.path.exists(mode):
            os.makedirs(mode)
        addres=(f"{files[:-4]}")
        if not os.path.exists(addres):
            os.makedirs(addres)
        for clf in names:
            shutil.move(f"{clf}_cl.bin",addres)
            shutil.move(f"{clf}_init.png",addres)
            shutil.move(f"{clf}_mid.png",addres)
            shutil.move(f"{clf}_out.png",addres)
        shutil.move(addres,mode)
    shutil.move(f"{mode}.txt",mode)

I started barbara.png
47.446 %
8.077 s


48.161 %
7.755 s


48.165 %
7.91 s


48.157 %
7.131 s


ValueError: Image must be 2D (grayscale, RGB, or RGBA).

In [10]:
shape

512

In [13]:
decryptedImage(files[:-4],shape,key,mode="ECB")

ValueError: Image must be 2D (grayscale, RGB, or RGBA).